# Lab 5: HuggingFace and Visual and Large Multimodal Models

This lab focuses on how to utilize Visual or Large Multimodal Models (LMM), as well as demonstrating the use of various HuggingFace models and pipelines. 

We divide the lab into five notebooks.  You should reset the kernel each time before running a new notebook, to free up the GPU memory.  Otherwise you might get out of memory errors.  

Note: The weights of these several models in this notebook are sourced from HuggingFace and have already been downloaded to **`/share/lab5`**, you can use them directly. 

We demonstrate these three models: 

1. Segment anything model (SAM) - this notebook
    The Segment Anything Model (SAM) produces high quality object masks from input prompts such as points or boxes, and it can be used to generate masks for all objects in an image.   
    (https://github.com/facebookresearch/segment-anything)

2. Stable-diffusion: see 2-sd.ipynb
    A latent text-to-image diffusion model.   
    (https://github.com/CompVis/stable-diffusion)  We are using version 1.5, but please feel free to try 2.1 version (also available on the servers.)

3. Fuyu - see 3-fuyu.ipynb
    A small version of the multimodal model.  
    The model is available on HuggingFace: (https://huggingface.co/adept/fuyu-8b) 

4. （optional）a text to video model - see 4-t2v.ipynb

5. [Your task] We have downloaed a set of models in **`/share/LLMs`**, your task is to create a notebook to run at least one of them.  (You can search for the documenation on Hugging Face Hub).  If you want to download your own model from HuggingFace, please do so on your laptop, and then upload onto the cluster (instead of downloading it directly on the cluster as we have very limited Internet bandwidth.)


## 5.1 Segment Anything

In [1]:
# We have already downloaded the model for you, and the file is at `/share/lab5/sam`

#!export HF_ENDPOINT=https://hf-mirror.com
#!huggingface-cli download --resume-download facebook/sam-vit-huge --local-dir your_path_of_sam

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
#import cv2
import gc
from PIL import Image
from transformers import pipeline

In [10]:
# open and show raw images
# you can examine the directory for the images

img_path = '/share/lab5/data/test3.jpg'
raw_image = Image.open(img_path)
raw_image.show()

In [4]:
# Create a mask-generation pipeline from the model /share/lab5/sam/, note the ending "/" in the path name

generator = pipeline("mask-generation", model="/share/lab5/sam/", device=0)


In [11]:
# Using the pipeline to generate the mask

outputs = generator(raw_image, points_per_batch=64)
masks = outputs["masks"]

In [17]:
# Examine the mask data structure
print(type(masks))
print(masks[0].shape)
masks[0]

In [6]:
## some utility functions to handle the masks

def show_mask(mask, ax, random_color=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    del mask
    gc.collect()

def show_masks_on_image(raw_image, masks):
  plt.imshow(np.array(raw_image))
  ax = plt.gca()
  ax.set_autoscale_on(False)
  for mask in masks:
    show_mask(mask, ax=ax, random_color=True)
  plt.axis("off")
  plt.show()
  del mask
  gc.collect()

In [7]:
show_masks_on_image(raw_image, masks)

In [8]:
# Another example 

img_path = '/share/lab5/data/test8.jpg'
raw_image = Image.open(img_path)
raw_image.show()

outputs = generator(raw_image, points_per_batch=64)
masks = outputs["masks"]
show_masks_on_image(raw_image, masks)

In [ ]:
#### Your Task ####
# upload another image of your choice and generate the mask for it
# show the entire mask

In [ ]:
#### Your Task ####
# show only the mask with the largest area
# hint: you need to examine the masks data structure to see how to calcluate the area.
# if there are tow masks with the same area size, show either one
